# Minimal Data Preprocessing — ID + Cleaned Text Only

This script converts company and job datasets into a **two-column format**:
- an **ID** column (e.g., `prsId` for companies, `jobId` for jobs), and
- a **cleaned text** column with all HTML removed.

Designed to be drop-in: if you already have your DataFrames (`df_firmen`, `df_jobs`),
skip the data loading step and directly use `to_two_columns`.

In [1]:
import re
import json
import pandas as pd
from pathlib import Path
from typing import Iterable, Union

# Data Loading (Optional)
Adjust file names / readers as needed.
If your DataFrames `df_firmen` and `df_jobs` are already in memory,
you can skip this entire section.

In [3]:
# Try JSON Lines first; fall back to normal JSON if needed.
def try_read_json_any(path: Union[str, Path]) -> pd.DataFrame:
    """
    Read a JSON Lines (.jsonl) file if possible; otherwise fall back to
    standard JSON (.json). Raises FileNotFoundError if the path doesn't exist.
    """
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"{path} not found. Please adjust the path.")
    try:
        # JSON Lines (one JSON object per line)
        return pd.read_json(path, lines=True, dtype=False)
    except ValueError:
        # Standard JSON (array or object)
        return pd.read_json(path, dtype=False)

# Example filenames (change as needed)
firmen_path = Path("companies.jsonl")  # or "companies.json"
jobs_path   = Path("jobs.jsonl")       # or "jobs.json"

# Try to load; if it fails, we simply print a message so the script can continue
try:
    df_firmen = try_read_json_any(firmen_path)
    df_jobs   = try_read_json_any(jobs_path)
    print("Loaded dataframes: df_firmen and df_jobs")
except Exception as e:
    print("Skipping data load (edit paths as needed). Error:", e)

Skipping data load (edit paths as needed). Error: companies.jsonl not found. Please adjust the path.


# Utilities
Helper functions for text processing:
 - `strip_html`: removes HTML tags and collapses whitespace
 - `value_to_text`: converts lists, dicts, or scalars into a text string
 - `concat_row_text`: concatenates all non-ID column values in one row into text

In [4]:

def strip_html(text: Union[str, float]) -> str:
    """Remove HTML tags and collapse whitespace. Robust to non-strings."""
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)
    # remove tags
    no_tags = re.sub(r"<[^>]+>", " ", text)
    # unescape common entities (basic replacements)
    no_entities = (no_tags
        .replace("&nbsp;", " ")
        .replace("&amp;", "&")
        .replace("&lt;", "<")
        .replace("&gt;", ">")
        .replace("&quot;", '"')
        .replace("&#39;", "'"))
    # collapse whitespace
    return re.sub(r"\s+", " ", no_entities).strip()

def value_to_text(val) -> str:
    """Convert scalars, lists, dicts to a compact text string."""
    if val is None:
        return ""
    if isinstance(val, (list, dict)):
        try:
            return json.dumps(val, ensure_ascii=False)
        except Exception:
            return str(val)
    return str(val)

def concat_row_text(row: pd.Series, exclude_cols: Iterable[str]) -> str:
    """Concatenate all non-ID column values into a single string."""
    parts = []
    for col, val in row.items():
        if col in exclude_cols:
            continue
        txt = value_to_text(val)
        if txt:
            parts.append(txt)
    return " ".join(parts)

# Transform to Two-Column Format

Convert a DataFrame into a minimal two-column format:
 - one column for the ID
 - one column for the cleaned text (all other fields concatenated and HTML-stripped).

In [5]:
def to_two_columns(df: pd.DataFrame, id_col: str, text_col_name: str = "text") -> pd.DataFrame:
    if id_col not in df.columns:
        raise KeyError(f"ID column '{id_col}' not found in DataFrame.")
    # Build raw text by concatenating all non-ID columns
    exclude = {id_col}
    raw_text = df.apply(lambda row: concat_row_text(row, exclude_cols=exclude), axis=1)
    # Clean HTML
    cleaned = raw_text.apply(strip_html)
    out = pd.DataFrame({id_col: df[id_col], text_col_name: cleaned})
    # Drop rows where text is empty after cleaning
    out = out[out[text_col_name].str.len() > 0].reset_index(drop=True)
    return out

# Example usage (uncomment when df_firmen / df_jobs are available)
# companies_two = to_two_columns(df_firmen, id_col="prsId")  # -> columns: prsId, text
# jobs_two      = to_two_columns(df_jobs,   id_col="jobId")  # -> columns: jobId, text

# Quick preview if dataframes are available
try:
    display(companies_two.head(3))
    display(jobs_two.head(3))
except NameError:
    print("Define df_firmen / df_jobs and run the transformation to preview the result.")

Define df_firmen / df_jobs and run the transformation to preview the result.


# Save Results (Optional)

Save the transformed two-column DataFrames to CSV or Parquet files.

In [ ]:
companies_two.to_parquet("companies_two_columns.parquet", index=False)
jobs_two.to_parquet("jobs_two_columns.parquet", index=False)

companies_two.to_csv("companies_two_columns.csv", index=False)
jobs_two.to_csv("jobs_two_columns.csv", index=False)